In [ ]:
%matplotlib inline
from IPython.display import SVG, Image

import os.path
import sys
import re
import itertools
import csv
import datetime
import pickle
import random
from collections import defaultdict, Counter
import gc

import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import seaborn as sns
import pandas as pd
import numpy as np
import scipy
import gensim
from sklearn.metrics import f1_score, classification_report, confusion_matrix, log_loss
from sklearn.model_selection import train_test_split
import gensim
from keras.preprocessing.sequence import skipgrams
import tensorflow as tf

In [ ]:
img_cnt = 0
def save_img():
    global img_cnt
    img_cnt += 1
    img_name = 'img_{0:03}.jpeg'.format(img_cnt)
    print(img_name)
    plt.savefig(img_name, pil_kwargs={'quality': 100})

In [ ]:
from numpy.random import default_rng
rng = default_rng(10)

def make_mat(nrow, ncol, from_row, to_row, ratio):
    nn = int(ncol*ratio)
    idx = np.arange(ncol)
    ret0 = np.zeros((nrow, ncol), dtype=int)
    for ii in np.arange(from_row, to_row):
        ret0[ii, rng.choice(idx, nn, replace=False)] = 1
    return ret0

# Create Data
---

In [ ]:
N_ROW = 1200

In [ ]:
mat1 = make_mat(N_ROW, 5, 0, 300, 0.6)
mat1

In [ ]:
mat1_1 = make_mat(N_ROW, 7, 0, 500, 0.6)
mat1_1

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(np.c_[mat1, mat1_1][:600,:].T)

In [ ]:
mat2 = make_mat(N_ROW, 9, 200, 500, 0.6)
mat2

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(np.c_[mat1, mat1_1, mat2][:600,:].T)

In [ ]:
mat3 = make_mat(N_ROW, 20, 200, 700, 0.6)
mat3

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(np.c_[mat1, mat1_1, mat2, mat3][:700,:].T)

In [ ]:
mat4 = make_mat(N_ROW, 15, 450, 700, 0.6)
mat4

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(np.c_[mat1, mat1_1, mat2, mat3, mat4][:700,:].T)

In [ ]:
mat5 = make_mat(N_ROW, 20, 450, 1000, 0.6)
mat5

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(np.c_[mat1, mat1_1, mat2, mat3, mat4, mat5].T)

In [ ]:
mat6 = make_mat(N_ROW, 10, 600, 1000, 0.6)
mat6

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(np.c_[mat1, mat1_1, mat2, mat3, mat4, mat5, mat6].T)

In [ ]:
mat7 = make_mat(N_ROW, 20, 800, 1200, 0.6)
mat7

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(np.c_[mat1, mat1_1, mat2, mat3, mat4, mat5, mat6, mat7].T)

In [ ]:
X = np.c_[mat1, mat1_1, mat2, mat3, mat4, mat5, mat6, mat7]
X.shape

In [ ]:
X_df = pd.DataFrame(X, dtype=int)
X_df.index = ['r'+ee.astype('str') for ee in (np.arange(X_df.shape[0])+1)]
X_df.columns = ['c'+ee.astype('str') for ee in np.arange(X_df.shape[1])+1]
#X_df['cls'] = ['cls'+str(ii) for ii in cls_user]
print(X_df.shape)
X_df.head()

In [ ]:
X_df.to_csv('sample_data.csv')

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(X.T)

# 元のデータに因子分析と主成分分析
---
* [apply factanal to sample012](https://www.kaggle.com/wordroid/apply-factanal-to-sample012)

In [ ]:
from sklearn import decomposition
pca = decomposition.PCA(n_components=2)

In [ ]:
pca.fit(X_df)
rscore = pca.transform(X)
print(rscore.shape)

In [ ]:
df = pd.DataFrame(rscore[:,:5])
sns.pairplot(df, markers='o', height=3.5, diag_kind='hist')
save_img()

In [ ]:
df = pd.DataFrame(pca.components_.T[:,:5])
sns.pairplot(df, markers='o', height=3.5, diag_kind='hist')
save_img()

In [ ]:
from sklearn import decomposition
fa = decomposition.FactorAnalysis(n_components=8)

In [ ]:
fa.fit(X_df)
rscore = fa.transform(X)
print(rscore.shape)

In [ ]:
pd.DataFrame(fa.components_.T)

In [ ]:
df = pd.DataFrame(rscore[:,:5])
sns.pairplot(df, markers='o', height=3.5, diag_kind='hist')
save_img()

In [ ]:
df = pd.DataFrame(fa.components_.T[:,:5])
sns.pairplot(df, markers='o', height=3.5, diag_kind='hist')
save_img()

# 特徴量を算出
---
* [lowcol sample012 fet](https://www.kaggle.com/wordroid/lowcol-sample012-fet)

# 相関（距離類似度）
---
* [lowcol sample012 cor](https://www.kaggle.com/wordroid/lowcol-sample012-cor)

# 因子分析
---
* [apply factanal to sample012 cor](https://www.kaggle.com/wordroid/apply-factanal-to-sample012-cor)
* ランク落ちする
* faで対応